In [2]:
from pyspark import SparkContext, SparkConf
import random
import numpy as np
import itertools

In [2]:
from pyspark import SparkContext, SparkConf
import random

if SparkContext._active_spark_context:
    SparkContext._active_spark_context.stop()


# Configuración de Spark
conf = SparkConf().setAppName("TriangleFinding").setMaster("local")
sc = SparkContext(conf=conf)

# Ejemplo de aristas del grafo como una lista de tuplas
edges = [
    (1, '11', 2),
    (2, '11', 3),
    (3, '11', 1),
    (3, '11', 4),
    (4, '11', 5),
    (5, '11', 3),
    # Agrega más aristas según sea necesario
]

# Crear un RDD a partir de la lista de aristas
edges_rdd = sc.parallelize(edges)

sc

24/06/18 22:41:35 WARN Utils: Your hostname, vserver resolves to a loopback address: 127.0.1.1; using 192.168.1.97 instead (on interface enp6s18)
24/06/18 22:41:35 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/18 22:41:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


<SparkContext master=local appName=TriangleFinding>

In [3]:
rdd = sc.parallelize([(1,11,2),(2,12,3),(3,13,1)])


Voy a empezar creando la función de hash. Entre algunas cosas que asumimos está que un nodo no puede tener una arista consigo mismo, entonces el número de nodos tiene que ver con el numero de aristas

In [4]:
rdd.count()

3

In [5]:
def func_obtener_nodos(rdd):
  nodos = []
  for elemento in rdd.collect():
    if elemento[0] not in nodos:
      nodos.append(elemento[0])

    if elemento[2] not in nodos:
      nodos.append(elemento[2])
  return nodos
func_obtener_nodos(rdd)

[1, 2, 3]

Implementa un programa en PySpark que entregue todos los tri´angulos (como tuplas de tres nodos)
en el grafo usando b
3
reducers, donde b es un par´ametro. Para esta primera parte puedes asumir que
tu grafo solo usa una etiqueta de arista (en el grafo de prueba, esa etiqueta corresponde al numero 11).

In [6]:
rdd.map(lambda x: (x[0], x[2], 1)).collect()

[(1, 2, 1), (2, 3, 1), (3, 1, 1)]

In [7]:
def hash_node(v, b):
  # Hasheamos los nodos, ej b_1 = h(n_1)
  return hash(v) % b

# Fase de mapeo
def map_triangles(arista):
  n_1, R, n_2 = arista
  b_1 = hash_node(n_1, b)

  b_2 = hash_node(n_2, b)

  resultado = []
  for b_3 in range(b):
    resultado.append(((b_1, b_2, b_3), (n_1, R, n_2)))
    resultado.append(((b_1, b_3, b_2), (n_1, R, n_2)))
    resultado.append(((b_3, b_1, b_2), (n_1, R, n_2)))

  return resultado



# Fase de Reduce
def reduce_triangles(llave, aristas):
  triangles = []

  aristas_dict = dict()

  for arista in aristas:

    n_1, R, n_2 = arista
    if n_1 not in aristas_dict:
      aristas_dict[n_1] = []
    aristas_dict[n_1].append((R, n_2))

  for n_1 in aristas_dict:
    for (R, n_2) in aristas_dict[n_1]:
      if n_2 in aristas_dict:
        for (s, n_3) in aristas_dict[n_2]:
          if n_3 in aristas_dict and any(n_4 == n_1 for (_, n_4) in aristas_dict[n_3]):
            triangles.append((n_1, n_2, n_3))
  return triangles

# Parámetro b
b = 2

# Fase de Map
mapped_rdd = edges_rdd.flatMap(lambda edge: map_triangles(edge))

# Particionamos la rdd para que se usen exactamente b^3 reducers

partitioned_rdd = mapped_rdd.partitionBy(b**3)

grouped_rdd = mapped_rdd.groupByKey()

triangles_rdd = grouped_rdd.flatMap(lambda key_value: reduce_triangles(key_value[0], list(key_value[1])))

# Acción final para recolectar los resultados
triangles = triangles_rdd.collect()

# Mostrar los triángulos encontrados
print(triangles)


[(1, 2, 3), (2, 3, 1), (3, 1, 2), (3, 4, 5), (4, 5, 3), (5, 3, 4)]


El enunciado no especifica si la matriz de relaciones es un pandas dataframe o una lista de listas, por lo que mi grupo asumió lo segundo

Asume ahora que recibes un subgrafo como tres arreglos: un arreglo A con las variables, otro L con los
tipos de aristas, y una matriz M de tama˜no |A| × |L| × |A| que tiene un uno en la posicion (x, R, y) si
y solo si (x, R, y) es una arista de tu subgrafo.

In [8]:
A = [0, 1, 2, 3] #Reemplazar por el arreglo de variables
L = ["R", "S", "T"] #Reemplazar por el arreglo de tipos de aristas

M = "" #Reemplazar por la matriz M

# Ej de M:
M = np.array([
    # R  S  T
    [[0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 0]], # x = 0
    [[0, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0]], # x = 1 # Chequear error (1,1)
    [[0, 0, 0, 0], [1, 0, 0, 0], [0, 1, 0, 0]], # x = 2
    [[0, 0, 1, 0], [0, 1, 0, 0], [0, 0, 0, 0]]  # x = 3
])

def matriz_a_aristas(A, L, M):
    aristas = []
    for x in range(len(M)):
        for y in range(len(M[0])):
            for r in range(len(M[0][0])):
              if M[x][y][r] == 1:
                  aristas.append((A[x], L[y], A[r]))
    return aristas

aristas = matriz_a_aristas(A, L, M)

print("Aristas: ", aristas)

if SparkContext._active_spark_context:
    SparkContext._active_spark_context.stop()


conf = SparkConf().setAppName("Subgrafos").setMaster("local[*]")
sc = SparkContext(conf=conf)

aristas_rdd2 = sc.parallelize(aristas)


# Ahora al igual que antes buscamos los patrones triangulares en el grafo
# Parámetro b
b = 2

# Fase de Map
mapped_rdd = aristas_rdd2.flatMap(lambda edge: map_triangles(edge))

# Particionamos la rdd para que se usen exactamente b^3 reducers

partitioned_rdd = mapped_rdd.partitionBy(b**3)

grouped_rdd = mapped_rdd.groupByKey()

triangles_rdd = grouped_rdd.flatMap(lambda key_value: reduce_triangles(key_value[0], list(key_value[1])))

# Acción final para recolectar los resultados
triangles = triangles_rdd.collect()

# Mostrar los triángulos encontrados
print(triangles)



Aristas:  [(0, 'R', 1), (0, 'S', 2), (1, 'S', 1), (1, 'T', 2), (2, 'S', 0), (2, 'T', 1), (3, 'R', 2), (3, 'S', 1)]
[(1, 1, 1), (1, 1, 2), (1, 2, 1), (2, 1, 1), (1, 1, 1), (0, 1, 2), (1, 2, 0), (2, 0, 1), (1, 1, 1), (1, 1, 1), (1, 1, 1), (1, 1, 1), (1, 1, 1), (1, 1, 1), (1, 1, 1), (1, 1, 1), (1, 1, 1), (1, 1, 1)]


# 2.4

In [5]:
# Configuración de Spark
if SparkContext._active_spark_context:
    SparkContext._active_spark_context.stop()

conf = SparkConf().setAppName("p2")
sc = SparkContext(conf=conf)


def hash_node(node, b):
    return node % b

def mapper_func(edge, b):
    n1, r, n2 = edge
    h_n1 = hash_node(n1, b)
    h_n2 = hash_node(n2, b)
    
    messages = []
    
    # (x,11,y) -> (n1, r, n2)
    for h in range(b):
        messages.append(((h_n1, h_n2, h, h), (n1, r, n2)))
    
    # (y,11,z) -> (n1, r, n2)
    for h in range(b):
        messages.append(((h, h_n1, h_n2, h), (n1, r, n2)))
    
    # (z,11,w) -> (n1, r, n2)
    for h in range(b):
        messages.append(((h, h, h_n1, h_n2), (n1, r, n2)))
    
    # (w,11,x) -> (n1, r, n2)
    for h in range(b):
        messages.append(((h_n2, h, h, h_n1), (n1, r, n2)))
    
    return messages

def reducer_func(key, edges):
    matches = []
    edges = list(edges)
    
    edge_dict = {}
    for n1, r, n2 in edges:
        if n1 not in edge_dict:
            edge_dict[n1] = []
        edge_dict[n1].append((r, n2))
    
    # Find patterns
    for n1, r1, n2 in edges:
        if n2 in edge_dict:
            for r2, n3 in edge_dict[n2]:
                if n3 in edge_dict:
                    for r3, n4 in edge_dict[n3]:
                        if n4 in edge_dict and any(r4 == 11 and n1_ == n1 for r4, n1_ in edge_dict[n4]):
                            matches.append((n1, n2, n3, n4))
    
    return matches

# The example graph edges given on the .md
graph_edges = [
    (1, 11, 2), (1, 11, 3), (2, 11, 3), (3, 11, 2),
    (3, 11, 4), (4, 11, 1), (4, 11, 2), (4, 11, 3),
    (4, 12, 5), (5, 12, 1), (5, 12, 2), (5, 12, 6)
]

b = 2

In [6]:
edges_rdd = sc.parallelize(graph_edges)
mapped_rdd = edges_rdd.flatMap(lambda edge: mapper_func(edge, b))
reduced_rdd = mapped_rdd.groupByKey().flatMap(lambda kv: reducer_func(kv[0], kv[1]))
matches = reduced_rdd.collect()

print('The following matches were found')
print('-------')
for match in matches:
    print(match)
print('-------')

The following matches were found
-------
(1, 2, 3, 4)
(2, 3, 2, 3)
(2, 3, 4, 1)
(2, 3, 4, 3)
(3, 2, 3, 2)
(3, 2, 3, 4)
(3, 4, 1, 2)
(3, 4, 3, 2)
(3, 4, 3, 4)
(3, 4, 5, 2)
(4, 1, 2, 3)
(4, 3, 2, 3)
(4, 3, 4, 3)
(4, 5, 2, 3)
(1, 2, 3, 4)
(2, 3, 2, 3)
(2, 3, 4, 1)
(2, 3, 4, 3)
(3, 2, 3, 2)
(3, 2, 3, 4)
(3, 4, 1, 2)
(3, 4, 3, 2)
(3, 4, 3, 4)
(3, 4, 5, 2)
(4, 1, 2, 3)
(4, 3, 2, 3)
(4, 3, 4, 3)
(4, 5, 2, 3)
(1, 2, 3, 4)
(2, 3, 2, 3)
(2, 3, 4, 1)
(2, 3, 4, 3)
(3, 2, 3, 2)
(3, 2, 3, 4)
(3, 4, 1, 2)
(3, 4, 3, 2)
(3, 4, 3, 4)
(3, 4, 5, 2)
(4, 1, 2, 3)
(4, 3, 2, 3)
(4, 3, 4, 3)
(4, 5, 2, 3)
(1, 2, 3, 4)
(2, 3, 2, 3)
(2, 3, 4, 1)
(2, 3, 4, 3)
(3, 2, 3, 2)
(3, 2, 3, 4)
(3, 4, 1, 2)
(3, 4, 3, 2)
(3, 4, 3, 4)
(3, 4, 5, 2)
(4, 1, 2, 3)
(4, 3, 2, 3)
(4, 3, 4, 3)
(4, 5, 2, 3)
(1, 2, 3, 4)
(2, 3, 2, 3)
(2, 3, 4, 1)
(2, 3, 4, 3)
(3, 2, 3, 2)
(3, 2, 3, 4)
(3, 4, 1, 2)
(3, 4, 3, 2)
(3, 4, 3, 4)
(3, 4, 5, 2)
(4, 1, 2, 3)
(4, 3, 2, 3)
(4, 3, 4, 3)
(4, 5, 2, 3)
(1, 2, 3, 4)
(2, 3, 2, 3)
(2, 3, 4, 1)
(2, 3, 4, 